You are given four Pandas data frames demand, var_cost, fix_cost, and cap containing the regional demand (thous. of cars), variable production costs (thous. $US), fixed production costs (thous. $US), and production capacity (thous. of cars). Two python lists loc, and size have also been created, containing the different locations, and the two types of plant capacities. All these variables have been printed to the console for your viewing. The code to initialize, and defined the decision variables has been completed for you.

It makes sense that x is continuous because the units of demand and plant capacity are in millions of cars which you can have fractions of millions of cars. Overall, you correctly created the needed variables for the capacitated plant location model.

In [ ]:
from pulp import *
import pandas as pd
from random import normalvariate

def run_pulp_model():
    # Initialize, and Define Decision Vars.
    model = LpProblem("Capacitated Plant Location Model", LpMinimize)

    #Define Decision Variables
    loc = ['USA', 'Germany', 'Japan', 'Brazil', 'India']
    size = ['Low_Cap','High_Cap']
    x = LpVariable.dicts("production_", [(i,j) for i in loc for j in loc],
                        lowBound=0, upBound=None, cat='Continous')
    y = LpVariable.dicts("plant_", 
                        [(i,s) for s in size for i in loc], cat='Binary')

    # Define objective function
    # Add random variation to simulate different costs
    model += (lpSum([fix_cost.loc[i,s] * y[(i,s)] 
                    for s in size for i in loc])
            + lpSum(([var_cost.loc[i,j] + normalvariate(0.5,0.5)) * x[(i,j)] 
                    for i in loc for j in loc]))

    # Define Constraints
    # Total production = Total Demand
    # Add random variation to simulate different demand (relative to the forecast)
    for j in loc:
        rd = normalvariate(o, demand.loc[j,'Dmd']*0.05)
        model += lpSum([x[(i,j)] for i in loc]) == (demand.loc[j,'Dmd'] + rd)

    # total production should be less than production capacity
    for i in loc:
        model += lpSum([x[(i,j)] for j in loc]) <= lpSum([cap.loc[i,s]*y[(i,s)] for s in size])

    model.solve()
    LpStatus[model.status]

    for v in model.variables():
        print(v.name, '=', v.varValue)

    # What is expected cost of this supply chain network model?
    value(model.objective)

    # If demand increases in a region, how much profit is needed to cover the costs of production and shipping to that region? Shadow price

    # Which regions still have production capacity for future demand increase? Slack

    k = [{'name': name, 'shadow price':c.pi, 'slack':c.slack}
        for name, c in model.constraints.items()]
    
    o = {}
    for i in loc:
        # optiomal solution shipment quantity
        o[i] = value(lpSum([x[(i,j)] for j in loc]))
    # optimal solution cost
    o['Obj'] = value(model.objective)

    return o

In [ ]:
# Simulate 100 different models
output = []
for i in range(100):
    output.append(run_pulp_model())

df = pd.DataFrame(output)

In [ ]:
# Histograms with results

import matplotlib.pyplot as plt
plt.title('Hist of Prod. at region E')
plt.hist(df['E'])
plt.show()